# Section 1: Introduction

**Sparse coding** is about transforming signals. Specifically, it is about representing signals as a linear combination of a sparse set (relatively few) of basis vectors.

Linear transforms are invertible transforms of the form $$ D\alpha = x$$ where $D \in \mathbb{R}^{n\times n}$ and $\alpha, x \in \mathbb{R}^n$. We can think of the vector $\alpha$ as representing $x$ in some new basis, i.e., an alternative representation of the signal.

With a redundant transformation, the representation $\alpha$ has a higher dimension than $x$. Thus, we have $D \in \mathbb{R}^{n\times m}$, $x \in \mathbb{R}^n$, and $\alpha \in \mathbb{R}^m$ where $m > n$. This is still a linear transformation, but we know from linear algebra that there is not necessarily a unique solution. 

This redundant representation still preserves invertibility, since $$ \begin{align} x & = D\alpha \\ & = D\left(D^{\dagger} x\right) \\ & = \left(DD^{\dagger}\right)x \\ & = x \end{align} $$

How to get a unique solution? We must pick some restrictions on $\alpha$. In sparse coding, we pick the *sparsest* such $\alpha$. This is a highly nonlinear task.

This is a task for data representation. 

Data is generally structured, and we want to exploit this structure.

**Mathematical models** are of utmost importance in studying data. A trade-off in modeling is simplicity versus reliability. For example, the JPEG compression algorithm relies on the hypothesis that image data is locally analyzable via the DCT; this is not a feasible assumption for all types of image data!

"Sparseland" Model assumption: every input patch of an image can be represented by a sparse linear combination of atoms from a given dictionary. Assume a dictionary of 256 atoms, each of which is an 8-by-8 pixel patch. Then each patch can be represented by a list of 256 numbers (this is the $\alpha$). But, we could simply record the $8\times 8 = 64$ pixel values and store less information. The only way this new representation will help us is if it is sparse, and contains less than 64 nonzero numbers. This is our Sparseland assumption (that this is possible).

Given a dictionary of atoms and a signal, how can we find the sparsest atom decomposition? We could check all combinations, but this would be a combinatorial explosion.

Instead, we will look to approximation algorithms to do this optimizations. One example is Iterative Reweighted Least Squares (IRLS).

*How to obtain a dictionary?* Learn them from a large set of target signals. This allows us to tune the model to our particular application. There is no universal dictionary which will work for all types of signals.



### Mathematical Warm-up

For linear systems $Ax=b$ with $n$ equations and $m>n$ unknowns, with $A$ full-rank, there are infinitely many solutions. As engineers, we need to suggest a meaningful unique solution.

For example, imagine subsampling an image to a quarter of its original dimension size. Recovering the original image from the subsampled image is such an **underdetermined linear system**.

To obtain a unique solution, we must choose some measure of quality, or penalty, and use this to rank our solutions.  $$ \min_x J\left(x\right) \; \text{such that} \; Ax=b $$

Here, $J\left(x\right)$ is a cost/penalty function. How should we choose $J$? This is our modeling choice and depends on the problem we want to solve. This is called **regularization**: the original problem has a singularity, but we are making it regular.

The most popular choice is $L_2$-based regularization, where $$ J\left(x\right) = \frac{1}{2}\left\| Bx\right\|_2^2 $$ for a given matrix $B$. This is popular because it is straight-forward mathematically, and it happens to lead to a closed-form solution. Under specific conditions on $B$, we have a unique solution.


But how to solve it? First, define the Lagrangian $$L\left(x,\lambda\right) = \frac{1}{2}\left\|Bx\right\|_2^2 + \lambda^T \left(Ax-b\right) $$ to incorporate the constraint. Next, set the gradient of $L$ with respect to $x$ equal to zero: $$\nabla L = B^TB x + A^T \lambda = 0 $$ to obtain $$ x = -\left(B^TB\right)^{-1}A^T \lambda $$ We introduced $\lambda$, so we must now get rid of it, using the constraint $Ax = b$: 
$$ \begin{align} b & = Ax \\ & = - A\left(B^TB\right)^{-1}A^T\lambda \end{align} $$ so that $$ \lambda = -\left(A\left(B^TB\right)^{-1}A^T\right)^{-1}b $$ Finally, the solution is
$$ x = \left(B^TB\right)^{-1}A^T\left(A\left(B^TB\right)^{-1}A^T\right)^{-1}b $$


The $P_1$ problem is $$ \min_x \left\| x \right\|_1 \; \text{such that} \; Ax=b $$ It will turn out that this regularizer leads to sparse solutions.

One simple property is that the solution set is bounded and convex. Why do we not expect a single solution to this min problem? Because the $L_1$ norm is not strictly convex. We can prove this property easily. Let $x_1$, $x_2$ be minimizers. Then if we take a convex combination, we get $$ \begin{align}A \left(tx_1 + \left(1-t\right)x_2 \right) & = tAx_1 + \left(1-t\right)Ax_2 \\ & = b \end{align} $$ So any convex combo of solutions is a solution; but is it a minimizer? $$\begin{align} \left\| tx_1 + \left(1-t\right)x_2\right\|_1 & \leq t \left\|x_1\right\|_1 + \left(1-t\right)\left\|x_2\right\|_1  \\ & = \left\| x_1 \right\| \end{align} $$ which is of course the minimum, since $x_1$ and $x_2$ are minimizers. But then the value of $J$ for any convex combo of solutions is less than or equal to (and thus equal to) the minimum, so they are also minimizers. Thus, a convex combo of solutions is again a solution: the solution set is convex.

Why is it bounded? Well for any two minimizers we have $$ \left\| x_1 \right\|_1 = \left\|x_2\right\|_1, $$ and so we have $$ \left\| x_1 - x_2 \right\|_1 \leq \left\| x_1 \right\|_1 + \left\|x_2\right\|_2 = 2 J_{\min} $$

Important theoretical point: why does $L_1$ promote sparse solutions?

>Let $A\in \mathbb{R}^{n\times m}$ with $n < m$. Then the set of solutions to the $P_1$ problem must contain a solution with at most $n$ non-zeros.

> Assume $x^*$ is a solution which has $k>n$ non-zeros. Then $Ax^*$ is a linear combination of $k$ columns from $A$, which equals $b$. We can rearrange the columns so that these are the first $k$ columns of $A$. Then taking the first $k$ columns of $A$, we have an $n\times k$ (sub)matrix with $n < k$, so these columns must be linearly dependent. Hence some column is a linear combination of the other columns.

> Let $h$ be a vector of length $m$ which is in the null space of $A$. Consider a caandidate new solution $$ x = x^* + \epsilon h $$ It is easy to show that this is still a solution to $Ax=b$. Also, for small enough $\epsilon$, we have $$ \left| \epsilon \right| < \min_{i\leq j\leq k} \frac{x_j^*}{h_j} \implies \text{sign}\left\{x^* + \epsilon h\right\} = \text{sign}\left\{x^*\right\}, $$ i.e., we do not change the signs of the original entries. In this case, we can write $$ \begin{align} \left\| x^* + \epsilon h \right\|_1 & = \sum_{j=1}^k \text{sign}\left\{x_j^*\right\}\left(x_j^* + \epsilon h_j\right) \\ & = \left\|x^*\right\|_1 + \epsilon h^T \text{sign}\left\{ x^* \right\} \end{align} $$

>If $h^T \text{sign}\left\{x^*\right\} $ is greater than OR less than 0, in either case we can show that we have $$\left\| x^* + \epsilon h\right\|_1 < \left\|x^*\right\|_1 $$ which violates our assumption of the optimality of $x^*$. Thus, we must have $h^T\text{sign}\left\{x^*\right\} = 0$. Thus, the $L_1$ norms of $x^*$ and $x^* + \epsilon h $ are the same.

> This means that we can choose $\epsilon$ to "null" one entry in $x^* + \epsilon h$ to obtain a solution with $k-1$ non-zeros. By proceeding this way, we can get $n$ non-zeros (at which point we must stop because the linear dependence of the submatrix columns is no longer guaranteed).



### How to actually solve the $P_1$ problem?

It can be framed as a linear program!

$$ \min_x c^T x \; \text{such that} \; Ax=b, x\geq 0 $$

It is a classic trick: split $x$ into its positive and negative entries: $$ x = u-v $$ where $x,u,v \in \mathbb{R}^n$ and $u,v \succeq 0 $. With the $L_1$ norm, we have $$\begin{align} \left\|x\right\|_1 & = \sum_{k=1}^m \left| u_k-v_k\right| \\ & = \sum_{k=1}^m \left( u_k + v_k\right) \\ & = 1^T u + 1^T v \\ & = \left[\begin{array}{cc} 1^T & 1^T \end{array} \right]\left[ \begin{array}{c} u \\ v \end{array}\right] \end{align} $$ Similarly, the linear system $Ax=b$ becomes $$ b = \left[ \begin{array}{cc} A & -A \end{array}\right] \left[ \begin{array}{c} u \\ v \end{array}\right] $$ and we have, of course, $$ \left[ \begin{array}{c} u \\ v \end{array}\right] \succeq 0 $$
These reformulations are the linear program representation of the problem.

### What about the $P_0$ problem?

The $L_0$ norm is $$ \left\|x_0\right\| = \lim_{p\rightarrow 0} \left\|x\right\|_p^p = \sum_{k=1}^m \mathbb{I}\left(x_k\right) $$ This is not a convex function, so this is not a valid norm. It satisfies non-negativity and the triangle-inequality, but not homogeneity.

The $P_0$ problem is, as expected, $$ \min_x \left\|x\right\|_0 \; \text{such that} \; Ax=b $$

Is there a unique solution? Under which conditions? Can we easily test optimality of a candidate? How quickly can we find the solution (algorithm)?


